In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import openai
import json
from tqdm import tqdm
import re


In [13]:
ECON_DATA_PATH = 'openstax_scraper/output_econ.json'
GENERATED_QA_PATH = 'generated_QA_pairs_econ.json'

In [ ]:
for i, q in enumerate(self_check_questions):
    # remove the empty newlines and first few numbers followed by a period in the question by regex
    clean_question = re.sub(r'^\s*\d+\.\s*', '', q['question'])
    print(f'{i}: {clean_question}')

In [ ]:
econ_data = json.load(open(ECON_DATA_PATH))
self_check_questions = []
keyterms = []
keyconceptsandsummary = []

for item in econ_data:
    if item['section_name'] == 'self-check-questions':
        self_check_questions.append(item)
    elif item['section_name'] == 'keyterms':
        keyterms.append(item)
    elif item['section_name'] == 'keyconceptsandsummary':
        keyconceptsandsummary.append(item)

print(len(self_check_questions))
print(len(keyterms))
print(len(keyconceptsandsummary))

In [ ]:
for i, q in enumerate(self_check_questions):
    # remove the empty newlines and first few numbers followed by a period in the question by regex
    clean_question = re.sub(r'^\s*\d+\.\s*', '', q['question'])
    print(f'{i}: {clean_question}')

In [ ]:
from KPDistorter import kpdistorter

client = openai.AzureOpenAI(api_version='2024-06-01')
kpd = kpdistorter.KeyPointDistorter(client, model='gpt-4o-mini', seed=42)

generated_QA_pairs = []

# count = 0
with open(GENERATED_QA_PATH, 'w') as file:
    for question in tqdm([self_check_questions[562]], desc="Processing questions"):
        choices, correct_answer, prompt_and_response = kpd.convert_to_MCQ_v2(question)
        local_dict = {
            'choices': choices,
            'correct_answer': correct_answer,
            'type': 'self-check-questions',
            'chapter_name': question['chapter_name'],
            'textbook_name': question['textbook_name'],
            'chapter_url': question['chapter_url'],
            'question': question['question'],
            'answer': question['answer'],
            'prompt_and_response': prompt_and_response,
            }
        generated_QA_pairs.append(local_dict)
        file.write(json.dumps(local_dict) + '\n')
        # if count > 10:
        #     break
        # count += 1


In [ ]:
error_case = self_check_questions[562]